In [1]:
from google.colab import files
uploaded = files.upload()

Saving SaaS-Sales.csv to SaaS-Sales.csv


In [2]:
!pip install pandas prophet matplotlib

In [3]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# Parsing and cleaning the CSV data
def parse_and_clean_data(file_path):
    # Reading CSV file from uploaded file in Colab
    df = pd.read_csv(file_path)

    # Converting Order Date to datetime using pandas
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

    # Dropping rows with invalid dates
    df = df.dropna(subset=['Order Date'])

    # Converting Sales and Discount to numeric, handling invalid entries
    df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')
    df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce')
    df = df.dropna(subset=['Sales', 'Discount', 'Customer ID'])

    # Calculating discounted sales
    df['Revenue'] = df['Sales'] * (1 - df['Discount'])

    return df

# Aggregating data for ARPU
def calculate_arpu(df):
    # Grouping by month and year
    df['Month'] = df['Order Date'].dt.to_period('M').dt.to_timestamp()

    # Aggregating total revenue and unique customers per month
    monthly_data = df.groupby('Month').agg({
        'Revenue': 'sum',
        'Customer ID': 'nunique'
    }).reset_index()

    # Calculating ARPU
    monthly_data['ARPU'] = monthly_data['Revenue'] / monthly_data['Customer ID']

    return monthly_data

# Preparing data for Prophet
def prepare_prophet_data(monthly_data):
    prophet_df = monthly_data[['Month', 'ARPU']].rename(columns={'Month': 'ds', 'ARPU': 'y'})
    return prophet_df

# Forecasting ARPU with Prophet
def forecast_arpu(prophet_df):
    model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    model.fit(prophet_df)

    # Creating future dates for 12 months
    future = model.make_future_dataframe(periods=12, freq='M')
    forecast = model.predict(future)

    return model, forecast

# Estimating LTV
def estimate_ltv(df, monthly_data):
    # Calculating average revenue per customer
    customer_revenue = df.groupby('Customer ID')['Revenue'].sum()
    avg_customer_revenue = customer_revenue.mean()

    # Assuming a 5% monthly churn rate
    churn_rate = 0.05
    avg_lifespan_months = 1 / churn_rate
    ltv = avg_customer_revenue * avg_lifespan_months

    return ltv, avg_customer_revenue, avg_lifespan_months

# Plotting historical and forecasted ARPU
def plot_arpu_forecast(prophet_df, forecast):
    plt.figure(figsize=(10, 6))
    plt.plot(prophet_df['ds'], prophet_df['y'], 'b-', label='Historical ARPU')
    plt.plot(forecast['ds'], forecast['yhat'], 'r--', label='Forecasted ARPU')
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='r', alpha=0.1, label='Confidence Interval')
    plt.title('Historical and Forecasted ARPU')
    plt.xlabel('Date')
    plt.ylabel('ARPU ($)')
    plt.legend()
    plt.grid(True)
    plt.savefig('arpu_forecast.png')
    plt.close()

# Main execution for Colab
# Upload the SaaS-Sales.csv file to Colab and specify the path
file_path = '/content/SaaS-Sales.csv'  # Adjust path if needed (e.g., after uploading to Colab)

df = parse_and_clean_data(file_path)

# Calculating ARPU
monthly_data = calculate_arpu(df)

# Preparing data for Prophet
prophet_df = prepare_prophet_data(monthly_data)

# Forecasting ARPU
model, forecast = forecast_arpu(prophet_df)

# Estimating LTV
ltv, avg_customer_revenue, avg_lifespan_months = estimate_ltv(df, monthly_data)

# Plotting the forecast
plot_arpu_forecast(prophet_df, forecast)

# Printing results
print(f"Average Customer Revenue: ${avg_customer_revenue:.2f}")
print(f"Estimated Customer Lifespan: {avg_lifespan_months:.1f} months")
print(f"Estimated LTV: ${ltv:.2f}")
print("ARPU Forecast saved as 'arpu_forecast.png'")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpp8di842z/bzsfokdq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp8di842z/6hl7s6mb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=61088', 'data', 'file=/tmp/tmpp8di842z/bzsfokdq.json', 'init=/tmp/tmpp8di842z/6hl7s6mb.json', 'output', 'file=/tmp/tmpp8di842z/prophet_modelsfqjyyet/prophet_model-20250519190233.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
19:02:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:02:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


Average Customer Revenue: $19945.64
Estimated Customer Lifespan: 20.0 months
Estimated LTV: $398912.87
ARPU Forecast saved as 'arpu_forecast.png'
